In [ ]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

In [1]:
import os
import pandas as pd
import sqlalchemy as sa
import sys

from dataframe import preprocess, rename_header_dataframe

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

import database as db
from models import DataSP

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Database connection

In [2]:
engine, session = db.connect()
engine.echo = False

2023-03-19 23:37:04,241 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-19 23:37:04,242 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 23:37:04,243 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-19 23:37:04,243 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 23:37:04,244 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-19 23:37:04,245 INFO sqlalchemy.engine.Engine [raw sql] {}


# Load CSV files

In [4]:
filename = '../csv/george_data.csv.csv'
df = pd.read_csv(filename, sep=';', low_memory=False, skipinitialspace=True)

## Adds 'yes' in the George column ('yes' are records marked as reliable by George))

In [8]:
count_records_marked_by_george = session.query(DataSP)\
    .filter(DataSP.george == True)\
    .count()

if count_records_marked_by_george == 0:
    for row in df.iterrows():
        get_value_column_george = row[1]['GEORGE']
        if get_value_column_george.lower() == 'sim':
            session.query(DataSP)\
                .filter(DataSP.seq == row[1]['seq'])\
                .update({'george': True}, synchronize_session=False)

2022-11-21 20:56:35,273 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scientific_name AS data_scientific_name, data.scientific_name_authorship AS data_scientific_name_authorship, data.identified_by AS data_identified_by, data.year_identified AS data_year_identified, data.month_identified AS data_month_identified, data.day_identified AS data_day_identified, data.type_status AS data_type_status, data.recorded_by AS data_recorded_by, data.record_number AS d

## Close connection

In [ ]:
session.close()
engine.dispose()